
**Data Preperation fo Instruction Fine Tuning** 

Today we will take the data set for our fake company using the synthetic data which has been cleansed as "data/fixed_generated_table_structures_with_original_data.csv"
and format it into instruction, input, output pairs, save it to 'output.csv'. The next step we will format it so it is in an understandable format for fine tuning.

We will be using the Alpaca format which is a model fine tuned on a LLama-2 model. You can find different formats fro different models online

Load the packages

In [23]:
!pip install scikit-learn

Import the relevant packages

In [24]:
import csv
import pandas as pd
from sklearn.model_selection import train_test_split

Load the cleansed synthetic data to be formatted and name the output file

In [25]:

# Define the input and output file paths
input_file = 'data/fixed_generated_table_structures_with_original_data.csv'
output_file = 'output.csv'


Choose an instruction prompt, this can be anything you like

In [26]:

# Define the instruction text
instruction = "You are a column description generator. Given the following examples of Source System Name, Source System Acronym, Table Name, Table Description, ColumnNames, ColumnAcronyms, DataTypes, Nullability predict the Column Description \n"


Load in the csv file and read it

In [27]:

# Open the input CSV file and read its contents
with open(input_file, 'r') as file:
    reader = csv.DictReader(file)
    data = list(reader)


Now combine the dataset into three columns Instruction, Input and the Output

In [28]:

# Open the output CSV file and write the reformatted data
with open(output_file, 'w', newline='') as file:
    fieldnames = ['Instruction', 'Input', 'Output']
    writer = csv.DictWriter(file, fieldnames=fieldnames)
    writer.writeheader()

    for row in data:
        input_data = f"{row['Source System Name']},{row['Source System Acronym']},{row['Table Name']},{row['Table Description']},{row['ColumnNames']},{row['ColumnAcronyms']},{row['DataTypes']},{row['Nullability']}"
        output_data = row['ColumnDescription']

        writer.writerow({
            'Instruction': instruction,
            'Input': input_data,
            'Output': output_data
        })

print("Reformatting complete. Output saved to", output_file)

Reformatting complete. Output saved to output.csv


Open the resulting file once verified its correct

In [29]:

df = pd.read_csv("output.csv")
df = df.fillna("")


Now we format this into 

**'### Instruction'**

**'### Input'**

**'### Response'**


strings (which is the format used to create Alpaca (Stanford early model)). 

In the case where there is no input the prompt contains only an instruction and responce only format as Instruction + Response

**'### Instruction'**

**'### Response'**

In [30]:

text_col = []
for _, row in df.iterrows():
    prompt = "Below is an instruction that describes a task, paired with are task that provides further context. Write a response that appropriately completes the request.\n\n"
    instruction = str(row["Instruction"])
    input_query = str(row["Input"])
    response = str(row["Output"])

    if len(input_query.strip()) == 0:
        text = prompt + "### Instruction:\n" + instruction + "\n### Response:\n" + response
    else:
        text = (
            prompt
            + "### Instruction:\n"
            + instruction
            + "\n### Input:\n"
            + input_query
            + "\n### Response:\n"
            + response
        )

    text_col.append(text)


The line df.loc[:, "text"] = text_col is assigning the values from the text_col list to a new column named "text" in the DataFrame df.

In [31]:

df.loc[:, "text"] = text_col
print(df.head())
df.to_csv("train.csv", index=False)


                                         Instruction  \
0  You are a column description generator. Given ...   
1  You are a column description generator. Given ...   
2  You are a column description generator. Given ...   
3  You are a column description generator. Given ...   
4  You are a column description generator. Given ...   

                                               Input  \
0  Exploration Data System,EDS,SeismicSurveyData_...   
1  Exploration Data System,EDS,SeismicSurveyData_...   
2  Exploration Data System,EDS,SeismicSurveyData_...   
3  Exploration Data System,EDS,SeismicSurveyData_...   
4  Exploration Data System,EDS,SeismicSurveyData_...   

                                      Output  \
0  Unique identifier for each seismic survey   
1                 Date of the seismic survey   
2                                              
3                                              
4    Depth of the seismic survey measurement   

                                     

We split the entire dataset df into training and temporary sets using train_test_split(df, test_size=0.3, random_state=42). This assigns 70% of the data (2,730 examples) to the training set and 30% (1,170 examples) to the temporary set.

In [32]:

# Split the data into training, validation, and testing sets
train_df, temp_df = train_test_split(df, test_size=0.3, random_state=42)


We split the temporary set temp_df into validation and testing sets using train_test_split(temp_df, test_size=0.5, random_state=42). This assigns 50% of the temporary set (585 examples) to the validation set and the remaining 50% (585 examples) to the testing set.

In [33]:

val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42)


With this 70-15-15 split, you'll have more data for training while still having a sufficient number of examples for validation and testing. This can help improve the model's performance and generalization, especially when dealing with a smaller dataset.

In [34]:

# Save the split datasets to separate CSV files
train_df.to_csv("train.csv", index=False)
val_df.to_csv("val.csv", index=False)
test_df.to_csv("test.csv", index=False)

Finally prepared the cleansed data into the format we require for fine tuning and we have split it into training, validation and testing sets, taking random samples for each.